In [1]:
#from google.colab import drive

In [2]:
#drive.mount('/content/drive')

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import hashlib
from scipy.stats import iqr
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# importing utility modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# importing machine learning models for prediction
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [4]:
#train=pd.read_csv("/content/drive/MyDrive/Datasets/train_data.csv")
train=pd.read_csv("pune.csv")
display(train)#display data

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,11-12-2008 00:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,10,31,72,0.0,1013,18,10,114,5,pune
1,11-12-2008 01:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,9,30,72,0.0,1013,19,10,125,4,pune
2,11-12-2008 02:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,8,29,72,0.0,1013,18,10,136,4,pune
3,11-12-2008 03:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,8,29,72,0.0,1013,18,10,147,4,pune
4,11-12-2008 04:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,7,23,68,0.0,1014,20,10,136,4,pune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116131,11-03-2022 19:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,10,6,28,0.0,1009,30,10,235,7,pune
116132,11-03-2022 20:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,14,7,29,0.0,1010,29,10,289,9,pune
116133,11-03-2022 21:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,12,9,30,0.0,1012,27,10,343,12,pune
116134,11-03-2022 22:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,14,8,31,0.0,1011,26,10,234,11,pune


In [5]:
train.describe()

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
count,116136.000000,116136.000000,116136.0,116136.000000,116136.000000,116136.000000,116136.000000,116136.000000,116136.000000,116136.000000,116136.000000,116136.00000,116136.000000,116136.000000,116136.000000,116136.000000,116136.000000,116136.000000,116136.000000
mean,30.977475,20.987394,0.0,11.136041,6.208927,46.087001,15.141722,26.170739,26.364383,25.304290,17.327823,31.43028,57.646001,0.231727,1009.731522,25.304720,9.409899,206.138863,9.397663
std,4.320621,2.701768,0.0,1.004464,0.921580,31.263676,6.093959,4.507061,4.355228,4.660732,8.147513,34.09457,25.316012,0.802657,3.580792,4.659629,1.593393,90.489701,4.775019
min,21.000000,11.000000,0.0,8.300000,4.000000,0.000000,-12.000000,10.000000,11.000000,10.000000,1.000000,0.00000,4.000000,0.000000,997.000000,11.000000,0.000000,0.000000,0.000000
25%,28.000000,19.000000,0.0,10.500000,5.000000,18.000000,10.000000,25.000000,25.000000,22.000000,11.000000,3.00000,36.000000,0.000000,1007.000000,22.000000,10.000000,106.000000,6.000000
50%,30.000000,21.000000,0.0,11.000000,6.000000,46.000000,16.000000,26.000000,26.000000,24.000000,16.000000,12.00000,58.000000,0.000000,1010.000000,24.000000,10.000000,252.000000,9.000000
75%,34.000000,23.000000,0.0,11.900000,7.000000,73.000000,21.000000,29.000000,29.000000,28.000000,22.000000,66.00000,82.000000,0.100000,1012.000000,28.000000,10.000000,274.000000,12.000000
max,43.000000,28.000000,0.0,13.600000,8.000000,100.000000,25.000000,44.000000,44.000000,43.000000,76.000000,100.00000,100.000000,25.700000,1021.000000,43.000000,10.000000,360.000000,40.000000


In [6]:
train.isnull().sum()

date_time            0
maxtempC             0
mintempC             0
totalSnow_cm         0
sunHour              0
uvIndex              0
moon_illumination    0
moonrise             0
moonset              0
sunrise              0
sunset               0
DewPointC            0
FeelsLikeC           0
HeatIndexC           0
WindChillC           0
WindGustKmph         0
cloudcover           0
humidity             0
precipMM             0
pressure             0
tempC                0
visibility           0
winddirDegree        0
windspeedKmph        0
location             0
dtype: int64

In [7]:
data = train
data[['Date','Time']] = data['date_time'].str.split(' ',n=1,expand=True)



In [8]:
data

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,Date,Time
0,11-12-2008 00:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,72,0.0,1013,18,10,114,5,pune,11-12-2008,00:00
1,11-12-2008 01:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,72,0.0,1013,19,10,125,4,pune,11-12-2008,01:00
2,11-12-2008 02:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,72,0.0,1013,18,10,136,4,pune,11-12-2008,02:00
3,11-12-2008 03:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,72,0.0,1013,18,10,147,4,pune,11-12-2008,03:00
4,11-12-2008 04:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,68,0.0,1014,20,10,136,4,pune,11-12-2008,04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116131,11-03-2022 19:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,28,0.0,1009,30,10,235,7,pune,11-03-2022,19:00
116132,11-03-2022 20:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,29,0.0,1010,29,10,289,9,pune,11-03-2022,20:00
116133,11-03-2022 21:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,30,0.0,1012,27,10,343,12,pune,11-03-2022,21:00
116134,11-03-2022 22:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,31,0.0,1011,26,10,234,11,pune,11-03-2022,22:00


In [9]:
data[['Date','Month']] = data['Date'].str.split('-',n=1,expand=True)
data[['Month','Year']] = data['Month'].str.split('-',n=1,expand=True)


In [10]:
#data.drop(['location','date_time'],axis=1,inplace=True)

In [11]:
data

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,Date,Time,Month,Year
0,11-12-2008 00:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,1013,18,10,114,5,pune,11,00:00,12,2008
1,11-12-2008 01:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,1013,19,10,125,4,pune,11,01:00,12,2008
2,11-12-2008 02:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,1013,18,10,136,4,pune,11,02:00,12,2008
3,11-12-2008 03:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,1013,18,10,147,4,pune,11,03:00,12,2008
4,11-12-2008 04:00,30,18,0,11.0,6,93,4:33 PM,5:13 AM,6:57 AM,...,1014,20,10,136,4,pune,11,04:00,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116131,11-03-2022 19:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,1009,30,10,235,7,pune,11,19:00,03,2022
116132,11-03-2022 20:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,1010,29,10,289,9,pune,11,20:00,03,2022
116133,11-03-2022 21:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,1012,27,10,343,12,pune,11,21:00,03,2022
116134,11-03-2022 22:00,35,23,0,11.8,7,56,12:40 PM,1:33 AM,6:46 AM,...,1011,26,10,234,11,pune,11,22:00,03,2022


In [12]:
data.drop(['location','date_time'],axis=1,inplace=True)

In [13]:
convert_dict = {'Date': int,
                'Month': int,
                'Year' : int
                }
 
data = data.astype(convert_dict)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116136 entries, 0 to 116135
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   maxtempC           116136 non-null  int64  
 1   mintempC           116136 non-null  int64  
 2   totalSnow_cm       116136 non-null  int64  
 3   sunHour            116136 non-null  float64
 4   uvIndex            116136 non-null  int64  
 5   moon_illumination  116136 non-null  int64  
 6   moonrise           116136 non-null  object 
 7   moonset            116136 non-null  object 
 8   sunrise            116136 non-null  object 
 9   sunset             116136 non-null  object 
 10  DewPointC          116136 non-null  int64  
 11  FeelsLikeC         116136 non-null  int64  
 12  HeatIndexC         116136 non-null  int64  
 13  WindChillC         116136 non-null  int64  
 14  WindGustKmph       116136 non-null  int64  
 15  cloudcover         116136 non-null  int64  
 16  hu

In [15]:
# train['year'] = train['date_time'].dt.year
# train['month'] = train['date_time'].dt.month
# train['day_of_year'] = train['date_time'].dt.dayofyear
# train['time'] = train['date_time'].dt.time

In [16]:
data.drop(['moonrise','moonset','sunrise','sunset'],axis=1,inplace=True)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116136 entries, 0 to 116135
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   maxtempC           116136 non-null  int64  
 1   mintempC           116136 non-null  int64  
 2   totalSnow_cm       116136 non-null  int64  
 3   sunHour            116136 non-null  float64
 4   uvIndex            116136 non-null  int64  
 5   moon_illumination  116136 non-null  int64  
 6   DewPointC          116136 non-null  int64  
 7   FeelsLikeC         116136 non-null  int64  
 8   HeatIndexC         116136 non-null  int64  
 9   WindChillC         116136 non-null  int64  
 10  WindGustKmph       116136 non-null  int64  
 11  cloudcover         116136 non-null  int64  
 12  humidity           116136 non-null  int64  
 13  precipMM           116136 non-null  float64
 14  pressure           116136 non-null  int64  
 15  tempC              116136 non-null  int64  
 16  vi

In [18]:
data['Time'].unique()

array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00',
       '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00',
       '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00',
       '21:00', '22:00', '23:00'], dtype=object)

In [19]:
Timeformat = {'00:00': 24,'01:00': 1,'02:00': 2,'03:00': 3,'04:00': 4,'05:00': 5,'06:00': 6,'07:00': 7,'08:00': 8,'09:00': 9,
              '10:00': 10,'11:00': 11,'12:00': 12,'13:00': 13,'14:00': 14,'15:00': 15,'16:00': 16,'17:00': 17,'18:00': 18,
              '19:00': 19,'20:00': 20,'21:00': 21,'22:00': 22,'23:00': 23,}
data['Time'] = data['Time'].map(Timeformat)

In [20]:
convert_dict = {'Date': int,
                'Month': int,
                'Year' : int,
                'Time' : int
                }
 
data = data.astype(convert_dict)

In [21]:
data

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,...,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,Date,Time,Month,Year
0,30,18,0,11.0,6,93,13,18,18,18,...,0.0,1013,18,10,114,5,11,24,12,2008
1,30,18,0,11.0,6,93,14,19,19,19,...,0.0,1013,19,10,125,4,11,1,12,2008
2,30,18,0,11.0,6,93,13,18,18,18,...,0.0,1013,18,10,136,4,11,2,12,2008
3,30,18,0,11.0,6,93,13,18,18,18,...,0.0,1013,18,10,147,4,11,3,12,2008
4,30,18,0,11.0,6,93,13,20,20,20,...,0.0,1014,20,10,136,4,11,4,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116131,35,23,0,11.8,7,56,9,28,28,30,...,0.0,1009,30,10,235,7,11,19,3,2022
116132,35,23,0,11.8,7,56,9,28,28,29,...,0.0,1010,29,10,289,9,11,20,3,2022
116133,35,23,0,11.8,7,56,11,26,26,27,...,0.0,1012,27,10,343,12,11,21,3,2022
116134,35,23,0,11.8,7,56,11,26,26,26,...,0.0,1011,26,10,234,11,11,22,3,2022


In [22]:
y1 = data['maxtempC']
x1 = data[['Date','Month','Year','Time']]

In [23]:
x1

,Date,Month,Year,Time
0,11,12,2008,24
1,11,12,2008,1
2,11,12,2008,2
3,11,12,2008,3
4,11,12,2008,4
...,...,...,...,...
116131,11,3,2022,19
116132,11,3,2022,20
116133,11,3,2022,21
116134,11,3,2022,22


In [24]:
y1

0         30
1         30
2         30
3         30
4         30
          ..
116131    35
116132    35
116133    35
116134    35
116135    35
Name: maxtempC, Length: 116136, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split
#splitting
x_train1,x_test1,y_train1,y_test1 = train_test_split(x1,y1,test_size=0.2,random_state=42)

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()

In [27]:
rf1.fit(x_train1,y_train1)

RandomForestClassifier()

In [28]:
from sklearn.metrics import accuracy_score
y_pred1 = rf1.predict(x_test1)
accuracy_score(y_test1,y_pred1)

0.9961253659376614

In [29]:
y_pred1

array([23, 39, 30, ..., 32, 25, 32], dtype=int64)

In [30]:
y2 = data['mintempC']
x2 = data[['Date','Month','Year','Time']]

In [31]:
from sklearn.model_selection import train_test_split
#splitting
x_train2,x_test2,y_train2,y_test2 = train_test_split(x2,y2,test_size=0.2,random_state=42)

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf2 = RandomForestClassifier()

In [33]:
rf2.fit(x_train2,y_train2)

RandomForestClassifier()

In [34]:
from sklearn.metrics import accuracy_score
y_pred2 = rf2.predict(x_test2)
accuracy_score(y_test2,y_pred2)

0.9978904770105046

In [35]:
y_pred2

array([20, 25, 21, ..., 23, 22, 22], dtype=int64)

In [39]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow.python'

In [38]:
# y3 = data['cloudcover']
# x3 = data[['Date','Month','Year','Time']]
# x_train3,x_test3,y_train3,y_test3 = train_test_split(x3,y3,test_size=0.2,random_state=42)
# rf3 = RandomForestClassifier()
# rf3.fit(x_train3,y_train3)
# y_pred3 = rf3.predict(x_test3)
# print(accuracy_score(y_test3,y_pred3))
# y_pred3

In [ ]:
y4 = data['humidity']
x4 = data[['Date','Month','Year','Time']]
x_train4,x_test4,y_train4,y_test4 = train_test_split(x4,y4,test_size=0.2,random_state=42)
rf4 = RandomForestClassifier()


In [ ]:
rf4.fit(x_train3,y_train3)


MemoryError: could not allocate 105906176 bytes

In [ ]:
y5 = data['precipMM']
x5 = data[['Date','Month','Year','Time']]
x_train5,x_test5,y_train5,y_test5 = train_test_split(x5,y5,test_size=0.2,random_state=42)
rf5 = RandomForestClassifier()


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
rf5.fit(x_train5,y_train5)
y_pred5 = rf5.predict(x_test5)
print(accuracy_score(y_test5,y_pred5))
y_pred5